# Main

In [1]:
import pandas as pd
import random

In [7]:
ds = pd.read_csv("./perfect_datasets/perfect_dataset_v2.csv")


In [8]:
ds_overview = pd.read_csv("./perfect_datasets/perfect_dataset_v1_overview.csv")


In [9]:
similarity = pd.read_csv("./perfect_datasets/perfect_datasets_v4.csv")


In [10]:
temp = pd.DataFrame()
temp["title"] = similarity["name"].str.lower().str.split()
temp["title"] = similarity["name"].apply(lambda x: x.replace(":", " :"))


In [11]:

for_sugg = pd.DataFrame()
for_sugg["title"] = similarity["name"].str.lower().str.split()
for_sugg["title"] = similarity["name"].apply(lambda x: x.replace("", ""))


In [12]:

movie_sugg = []
c = 0

def recommend(movie):
    global c
    global movie_sugg
    if(ds[ds["title"].str.lower() == movie.lower()].empty):
        for i in temp["title"]:
            tmp = i.split(",")
            movi = tmp[0].lower().split()
            if(movie.lower() in movi) and c==0:
                movie_sugg.append(tmp[0].replace(" :", ":"))
            if(movie.lower() in movi) and c==1:
                movie_sugg = (tmp[0].replace(" :", ":"))
        print("did you mean: ", movie_sugg, end="\n\n") if c<=1 else print("No movie found")
        c += 1
        return
    elif(movie == ""):
        print("Enter a movie name")
        return
    
    for_sugg["title"] = for_sugg["title"].apply(lambda x: x.split(","))
    
    index = ds[(ds["title"].str.lower()) == movie.lower()].index[0]

    moviess = for_sugg["title"][index][1:]
    rndm = []
    for i in range(0,5):
        rd = random.randint(1,10)
        rndm.append(rd if rd not in rndm else random.randint(1,10))
    for i in rndm:
        index2 = ds[(ds["title"].str.lower()) == moviess[i].lower()].index[0]
        print("Movie Id:",ds.iloc[index2]["imdb_id"])
        print("Movie Name:",moviess[i].replace(" :", ":"))
        print("Movie Overview:",ds_overview.iloc[index2]["overview"], end="\n\n")


In [13]:

inp = str(input("Enter the movie name: "))
recommend(inp)

if movie_sugg != []:
        movie_re_input = str(input("Enter the movie name: "))
        recommend(movie_re_input)


Movie Id: tt0053242
Movie Name: Sapphire
Movie Overview: Two Scotland Yard detectives (Nigel Patrick and Michael Craig) investigate the murder of a young woman of mixed race who had been passing for white. As they interview a spate of suspects -- including the girl's white boyfriend and his disapproving parents -- the investigators wade through a stubbornly entrenched sludge of racism and bigotry. Director Basil Dearden won a British Academy Award for his deft, sensitive hand at the helm.

Movie Id: tt0286493
Movie Name: The Badge
Movie Overview: A sheriff (Thornton) begins an investigation into the death of a local transsexual after hearing that high ranking politicians may have been involved. Although he is homophobic, his investigation causes him to be rejected by others, forcing him to seek help from the people he once despised.

Movie Id: tt0053242
Movie Name: Sapphire
Movie Overview: Two Scotland Yard detectives (Nigel Patrick and Michael Craig) investigate the murder of a young 

## Chintan TfidfVectorizer

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from difflib import get_close_matches  # Python's difflib module for finding close matches

# Load data
ds = pd.read_csv("./perfect_datasets/perfect_dataset_v2.csv")

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(ds['title'])

# Calculate the cosine similarity between movies
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

def find_similar_movies(movie_name):
    similar_movies = []
    for title in ds['title']:
        similar_movies.extend(get_close_matches(movie_name, [title], n=5, cutoff=0.6))
    return list(set(similar_movies))

def recommend(movie_name):
    movie_name = movie_name.lower()
    
    # Check if the exact movie name exists
    exact_match = ds[ds['title'].str.lower() == movie_name]
    
    if not exact_match.empty:
        idx = exact_match.index[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        # sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:]  # Get the top 5 most similar movies
        movie_indices = [i[0] for i in sim_scores]
        recommended_movies = ds['title'].iloc[movie_indices]
        print("Recommended Movies:")
        print(recommended_movies)
    else:
        print("Movie not found. Did you mean:")
        similar_movies = find_similar_movies(movie_name)
        print(similar_movies)

# User input
inp = input("Enter a movie name: ")
recommend(inp)


Recommended Movies:
1       Pirates of the Caribbean: At World's End
2                                        Spectre
3                          The Dark Knight Rises
4                                    John Carter
5                                   Spider-Man 3
                          ...                   
4504                                      Cavite
4505                                   Newlyweds
4506                   Signed, Sealed, Delivered
4507                            Shanghai Calling
4508                           My Date with Drew
Name: title, Length: 4508, dtype: object
